In [1]:
import geopandas as gpd
import pandas as pd
import boto3
import os

In [2]:
bucket_name = 'opera-calval-database-dswx'
private_bucket_name = 'opera-calval-database-dswx-private'

In [3]:
local_planet_dir = './downloads/crop_ver2/'

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [5]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')


In [6]:
#test = collocation.head(100)
#for row in test.iterrows():
#    print(row[1].pl_ID)

In [7]:
fileList = os.listdir(local_planet_dir)
print(fileList)

['20210925_191708_94_2408_3B_AnalyticMS_SR_8b.tif', '20211027_151445_21_2231_3B_AnalyticMS_SR_8b.tif', '20211030_142613_41_227b_3B_AnalyticMS_SR_8b.tif', '20210926_072414_71_2233_3B_AnalyticMS_SR_8b.tif', '20210901_175205_71_2426_3B_AnalyticMS_SR_8b.tif', '20211014_152910_49_2459_3B_AnalyticMS_SR_8b.tif', '20211005_135201_42_227e_3B_AnalyticMS_SR_8b.tif', '20211028_045455_02_2459_3B_AnalyticMS_SR_8b.tif', '20210924_082025_48_2424_3B_AnalyticMS_SR_8b.tif', '20211011_155455_52_2262_3B_AnalyticMS_SR_8b.tif', '20211010_080351_47_2458_3B_AnalyticMS_SR_8b.tif', '20210906_183639_52_240c_3B_AnalyticMS_SR_8b.tif', '20210914_051809_10_2455_3B_AnalyticMS_SR_8b.tif', '20211023_034107_36_2223_3B_AnalyticMS_SR_8b.tif', '20211016_081255_90_2420_3B_AnalyticMS_SR_8b.tif', '20211008_054349_78_241d_3B_AnalyticMS_SR_8b.tif', '20211010_003854_61_241d_3B_AnalyticMS_SR_8b.tif', '20211008_073033_21_245c_3B_AnalyticMS_SR_8b.tif', '20210918_135626_99_245f_3B_AnalyticMS_SR_8b.tif', '20211025_085849_59_241b_3B_An

In [8]:
#fileList = fileList[0:1]
#print(fileList)
#restart_ID = '20210910_002538_99_2206'
#run = False
for file in fileList:
    if file[-3:] != 'tif':
        continue
        
    ID = file.split('_3B_Analytic')[0]
    #if ID == restart_ID:
    #    run = True
    
    #if run == False:
    #    continue
    print('Adding Image ID: '+ID)
    print('\tBuilding Image Table Entry')
    search = collocation[(collocation.pl_ID == ID)].sort_values('hls_pl_AOI_intersect',ascending=False)
    search = search.iloc[[0]]
    site = search.chip_ID.iloc[0]
    if site != '4_37':
        continue
    s3_key_image = 'data/'+site+'/'+ID+'/'+file
    new_image = search.drop(columns=['hls_Date','hls_Cloud_Cover','hls_Footprint','hls_AOI_intersect','hls_pl_AOI_intersect','chip_rID'])
    new_image = new_image.rename(columns = {'hls_ID':'collocated_dswx','pl_Date':'datetime','pl_ID':'image_name','pl_Cloud_Cover':'cloud_cover','pl_AOI_intersect':'site_coverage','chip_ID':'site_name'})
    new_image['s3_bucket'] = private_bucket_name
    new_image['s3_key_image'] = s3_key_image
    #print(new_image.head())
    obj = s3.Object(bucket_name,'image.geojson')
    image_data = obj.get()['Body']
    image_gdf = gpd.read_file(image_data)
    
    #print(image_gdf)
    
    
    print('\tUploading File')
    result = s3_client.list_objects_v2(Bucket=private_bucket_name, Prefix=s3_key_image)
    
    if 'Contents' in result:
        print("\tImage file already exists in bucket, overwriting existing file")
        response = s3_client.upload_file(local_planet_dir+file,private_bucket_name,s3_key_image)
    else:
        response = s3_client.upload_file(local_planet_dir+file,private_bucket_name,s3_key_image)
        #print('\tUpload manually skipped')
    
    print('\tUpdating Image Table')
    if 'image_name' in image_gdf.columns:
        if len(image_gdf[image_gdf.image_name==new_image.image_name.iloc[0]]) != 0:
            print('\timage_name: '+new_image.image_name.iloc[0]+' is already in image table')
            image_gdf_upd = image_gdf[image_gdf.image_name != new_image.image_name.iloc[0]]
        else:
            print('\tAdding new row to table')
    image_gdf_upd = image_gdf.append(new_image,ignore_index=True)
    image_upd_bytes = bytes(image_gdf_upd.to_json(drop_id=True).encode('UTF-8'))
    s3object = s3.Object(bucket_name,'image.geojson')
    s3object.put(Body=image_upd_bytes)
    print('\tCompleted')

Adding Image ID: 20210925_191708_94_2408
	Building Image Table Entry
Adding Image ID: 20211027_151445_21_2231
	Building Image Table Entry
Adding Image ID: 20211030_142613_41_227b
	Building Image Table Entry
Adding Image ID: 20210926_072414_71_2233
	Building Image Table Entry
Adding Image ID: 20210901_175205_71_2426
	Building Image Table Entry
Adding Image ID: 20211014_152910_49_2459
	Building Image Table Entry
Adding Image ID: 20211005_135201_42_227e
	Building Image Table Entry
Adding Image ID: 20211028_045455_02_2459
	Building Image Table Entry
Adding Image ID: 20210924_082025_48_2424
	Building Image Table Entry
	Uploading File
	Image file already exists in bucket, overwriting existing file
	Updating Image Table
	image_name: 20210924_082025_48_2424 is already in image table
	Completed
Adding Image ID: 20211011_155455_52_2262
	Building Image Table Entry
Adding Image ID: 20211010_080351_47_2458
	Building Image Table Entry
Adding Image ID: 20210906_183639_52_240c
	Building Image Table En

Adding Image ID: 20211014_083427_44_2408
	Building Image Table Entry
Adding Image ID: 20211006_080646_19_2464
	Building Image Table Entry
Adding Image ID: 20211012_083209_57_245c
	Building Image Table Entry
Adding Image ID: 20211003_075037_07_2458
	Building Image Table Entry
Adding Image ID: 20211024_075820_52_242d
	Building Image Table Entry
Adding Image ID: 20210924_081239_92_222b
	Building Image Table Entry
Adding Image ID: 20210928_072802_36_2463
	Building Image Table Entry
Adding Image ID: 20210924_133400_47_2450
	Building Image Table Entry
Adding Image ID: 20211018_085512_78_2413
	Building Image Table Entry
Adding Image ID: 20210906_012332_35_2449
	Building Image Table Entry


In [29]:
obj = s3.Object(bucket_name,'image.geojson')
image_data = obj.get()['Body']
image_gdf = gpd.read_file(image_data)

#image_gdf = image_gdf[pd.notnull(image_gdf.image_name)]
image_gdf.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
0,0.0,HLS.S30.T22KGE.2021282T133229.v2.0,2021-10-09T12:39:44,20211009_123944_46_2212,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/2_13/20211009_123944_46_2212/20211009_123...,82.409823,2_13,0.040089,"POLYGON ((-49.22010 -18.47171, -49.25746 -18.6..."
1,0.0,HLS.S30.T55TCJ.2021271T010651.v2.0,2021-09-28T01:06:12,20210928_010612_93_2424,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_5/20210928_010612_93_2424/20210928_0106...,14.555879,4_5,0.006154,"POLYGON ((144.94479 43.97568, 144.88627 43.782..."
2,0.0,HLS.L30.T41TMN.2021301T064037.v2.0,2021-10-28T05:51:09,20211028_055109_45_245f,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/2_6/20211028_055109_45_245f/20211028_0551...,98.218700,2_6,0.034358,"POLYGON ((62.89599 47.37032, 62.83030 47.17890..."
3,0.0,HLS.L30.T13VDF.2021285T180913.v2.0,2021-10-12T17:35:55,20211012_173555_20_2276,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_20/20211012_173555_20_2276/20211012_173...,64.450386,4_20,0.023128,"POLYGON ((-105.67707 59.39304, -105.77853 59.2..."
4,0.0,HLS.S30.T47VPG.2021261T042711.v2.0,2021-09-18T04:25:59,20210918_042559_64_2419,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/2_24/20210918_042559_64_2419/20210918_042...,97.830115,2_24,0.004040,"POLYGON ((101.69577 60.00023, 101.58257 59.812..."


In [27]:
#image_upd_bytes = bytes(image_gdf.to_json(drop_id=True).encode('UTF-8'))
#s3object = s3.Object(bucket_name,'image.geojson')
#s3object.put(Body=image_upd_bytes)